In [ ]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import rc, rcParams

%matplotlib inline

rc('text', usetex=True)
font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 14}
rc('font', **font)
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
model_col = []
solver_col = []
loss_col = []
mota_col = []
motp_col = []
prec_col = []
rec_col = []

for results_file in pathlib.Path('./eval').rglob('eval_results.csv'):
    model_dir = results_file.parent.parent
    solver_dir = model_dir.parent
    loss_dir = solver_dir.parent

    model = model_dir.stem
    solver = solver_dir.stem
    loss = loss_dir.stem

    df = pd.read_csv(str(results_file), index_col=0)
    overall_scores = df.iloc[-1]

    model_col.append(model)
    solver_col.append(solver)
    loss_col.append(loss)

    mota_col.append(overall_scores['mota'])
    motp_col.append(overall_scores['motp'])
    prec_col.append(overall_scores['precision'])
    rec_col.append(overall_scores['recall'])

df_orig = pd.DataFrame(data={
    'model': model_col,
    'solver': solver_col,
    'loss': loss_col,
    'mota': mota_col,
    'motp': motp_col,
    'prec': prec_col,
    'rec': rec_col,
})

df_orig

In [ ]:
df = df_orig[
    (df_orig['model'] == '0040000') |
    (df_orig['model'] == '0050000') |
    (df_orig['model'] == '0060000')
]
df

In [ ]:
def plot_tracker_loss_solver_comparison(
    df, x_col, y_col, x_label, y_label, x_units='\%', y_units='\%'
):
    def _build_axis_label(text, units=None):
        label = rf'$\textbf{{{text}}}$'
        if units is not None:
            label += f' [{units}]'
        return label

    fig, ax = plt.subplots(figsize=(16, 8), nrows=1, ncols=1)

    base_colors =  list(mcolors.BASE_COLORS.keys())
    unique_models = df['model'].unique().tolist()

    size = 30 ** 2
    for solver, group_solver_df in df.groupby(['solver']):
        if solver == 'orig':
            facecolors = 'none'
        else:
            facecolors = edgecolors = None
        
        for loss, group_loss_df in group_solver_df.groupby(['loss']):
            if loss == 'none':
                marker = 'o'
            elif loss == 'contr':
                marker = 'X'
            else:  # triplet
                marker = '^'
        
            for model, group_model_df in group_loss_df.groupby(['model']):
                xs, ys = group_model_df[x_col], group_model_df[y_col]    
                
                curr_color = base_colors[unique_models.index(model)]
                label = rf'${solver}, {loss}, {str(int(model))}$'
                if facecolors == 'none':
                    color = None
                    edgecolors = curr_color
                else:
                    color = curr_color
                
                ax.scatter(
                    xs, ys, s=size, c=color, label=label, marker=marker,
                    facecolors=facecolors, edgecolors=edgecolors, linewidth=4,
                    alpha=0.7
                )
    
    x_label_formatted = _build_axis_label(x_label, x_units)
    y_label_formatted = _build_axis_label(y_label, y_units)
    
    ax.set_title(
        r'$\textbf{Tracker Performance Comparison --- ' + 
        r'Training Iterations vs. Loss vs. Solver}$'
    )
    ax.set_xlabel(x_label_formatted)
    ax.set_ylabel(y_label_formatted)
    ax.legend(loc='lower right', prop={'size': 23})

    return fig

fig = plot_tracker_loss_solver_comparison(df, 'mota', 'motp', 'MOTA', 'MOTP')
fig.savefig('tracker_cmp_plot.png', dpi=300)